In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12,6)

In [10]:
orders = pd.read_csv("/Order_semicolon.csv", delimiter=";")
customers = pd.read_csv("/Customer_semicolon.csv", delimiter=";")
campaigns = pd.read_csv("/Campaign_semicolon.csv", delimiter=";")
campaign_queue = pd.read_csv("/CampaignQueue_semicolon.csv", delimiter=";")

In [11]:
orders["totalAmount"] = pd.to_numeric(orders["totalAmount"], errors="coerce")
orders["takeOutTimeInSeconds"] = pd.to_numeric(orders["takeOutTimeInSeconds"], errors="coerce")
orders["preparationTime"] = pd.to_numeric(orders["preparationTime"], errors="coerce")
orders["tempo_retirada_min"] = orders["takeOutTimeInSeconds"].fillna(orders["preparationTime"]) / 60
orders["createdAt"] = pd.to_datetime(orders["createdAt"], errors="coerce", dayfirst=True)


In [12]:
ranking_empresas = (
    orders.groupby("companyId")
    .agg(
        pedidos=("id", "count"),
        mediana_tempo=("tempo_retirada_min", "median"),
        p90_tempo=("tempo_retirada_min", lambda x: x.quantile(0.9)),
        ticket_medio=("totalAmount", "mean")
    )
    .reset_index()
    .sort_values("mediana_tempo")
)


In [13]:
ranking_plataformas = (
    orders.groupby("engineName")
    .agg(
        pedidos=("id", "count"),
        ticket_medio=("totalAmount", "mean"),
        mediana_tempo=("tempo_retirada_min", "median")
    )
    .reset_index()
    .sort_values("pedidos",ascending=False)
)


In [14]:
pedidos_dia = (
    orders.groupby(orders["createdAt"].dt.date)
    .agg(
        pedidos=("id", "count"),
        receita_total=("totalAmount", "sum"),
        ticket_medio=("totalAmount", "mean")
    )
    .reset_index()
)
